In [3]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

In [5]:
# source text
data = """ Piford Technologies is a leading Software Development Company\n
Piford Technologies provide trainings to working professionals and students\n
We are product based and service based company\n
we have one of our office in IT Park, Mohali\n """

In [10]:
mytokenizer = Tokenizer()
mytokenizer.fit_on_texts([data])
total_words = len(mytokenizer.word_index) + 1


In [8]:
# integer encode text
tokenizer = Tokenizer()
tokenizer.fit_on_texts([data])
encoded_data= tokenizer.texts_to_sequences([data])[0]
encoded_data

[1,
 2,
 7,
 8,
 9,
 10,
 11,
 3,
 1,
 2,
 12,
 13,
 14,
 15,
 16,
 4,
 17,
 5,
 18,
 19,
 6,
 4,
 20,
 6,
 3,
 5,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29]

In [11]:
mytokenizer.word_index

{'piford': 1,
 'technologies': 2,
 'company': 3,
 'and': 4,
 'we': 5,
 'based': 6,
 'is': 7,
 'a': 8,
 'leading': 9,
 'software': 10,
 'development': 11,
 'provide': 12,
 'trainings': 13,
 'to': 14,
 'working': 15,
 'professionals': 16,
 'students': 17,
 'are': 18,
 'product': 19,
 'service': 20,
 'have': 21,
 'one': 22,
 'of': 23,
 'our': 24,
 'office': 25,
 'in': 26,
 'it': 27,
 'park': 28,
 'mohali': 29}

In [13]:
my_input_sequences = []
for line in data.split('\n'):
    #print(line)
    token_list = mytokenizer.texts_to_sequences([line])[0]
    #print(token_list)
    for i in range(1, len(token_list)):
        my_n_gram_sequence = token_list[:i+1]
        #print(my_n_gram_sequence)
        my_input_sequences.append(my_n_gram_sequence)
        #print(input_sequences)

In [14]:
max_sequence_len = max([len(seq) for seq in my_input_sequences])
input_sequences = np.array(pad_sequences(my_input_sequences, maxlen=max_sequence_len, padding='pre'))


In [15]:
input_sequences[1]

array([0, 0, 0, 0, 0, 0, 0, 1, 2, 7])

In [16]:
X = input_sequences[:, :-1]
y = input_sequences[:, -1]

In [17]:
X[1]

array([0, 0, 0, 0, 0, 0, 0, 1, 2])

In [18]:
y = np.array(tf.keras.utils.to_categorical(y, num_classes=total_words))

In [19]:
y[1]

array([0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)

In [20]:
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))
model.add(LSTM(150))
model.add(Dense(total_words, activation='softmax'))
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 9, 100)            3000      
                                                                 
 lstm (LSTM)                 (None, 150)               150600    
                                                                 
 dense (Dense)               (None, 30)                4530      
                                                                 
Total params: 158,130
Trainable params: 158,130
Non-trainable params: 0
_________________________________________________________________
None


In [21]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=100, verbose=1)

Epoch 1/100
1/1 [==============================] - 5s 5s/step - loss: 3.4016 - accuracy: 0.0000e+00
Epoch 2/100
1/1 [==============================] - 0s 18ms/step - loss: 3.3934 - accuracy: 0.0645
Epoch 3/100
1/1 [==============================] - 0s 50ms/step - loss: 3.3852 - accuracy: 0.1613
Epoch 4/100
1/1 [==============================] - 0s 17ms/step - loss: 3.3767 - accuracy: 0.1613
Epoch 5/100
1/1 [==============================] - 0s 16ms/step - loss: 3.3677 - accuracy: 0.1613
Epoch 6/100
1/1 [==============================] - 0s 17ms/step - loss: 3.3578 - accuracy: 0.1935
Epoch 7/100
1/1 [==============================] - 0s 20ms/step - loss: 3.3469 - accuracy: 0.1935
Epoch 8/100
1/1 [==============================] - 0s 14ms/step - loss: 3.3346 - accuracy: 0.1613
Epoch 9/100
1/1 [==============================] - 0s 16ms/step - loss: 3.3205 - accuracy: 0.1290
Epoch 10/100
1/1 [==============================] - 0s 16ms/step - loss: 3.3041 - accuracy: 0.1290
Epoch 11/100
1/1 

In [24]:
input_text = " Piford "
predict_next_words= 6

for _ in range(predict_next_words):
    token_list = mytokenizer.texts_to_sequences([input_text])[0]
    print(token_list)
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
    predicted = np.argmax(model.predict(token_list), axis=-1)
    output_word = ""
    for word, index in mytokenizer.word_index.items():
        if index == predicted:
            output_word = word
            break
    input_text += " " + output_word

print(input_text)

[1]
1/1 [==============================] - 0s 49ms/step
[1, 2]
1/1 [==============================] - 0s 48ms/step
[1, 2, 7]
1/1 [==============================] - 0s 50ms/step
[1, 2, 7, 8]
1/1 [==============================] - 0s 48ms/step
[1, 2, 7, 8, 9]
1/1 [==============================] - 0s 47ms/step
[1, 2, 7, 8, 9, 10]
1/1 [==============================] - 0s 46ms/step
 Piford  technologies is a leading software development
